In [17]:
import os
from pyprojroot import here
os.environ["HF_HOME"] = str(here("cache/huggingface"))
import lightning as L
from importlib import reload
import sys
sys.path.append(str(here()))
from models import model_definitions as MD
import torch
import pandas as pd
reload(MD)
torch.set_float32_matmul_precision('medium')
data_module_3 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_3.parquet")))
data_module_7 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_7.parquet")))
data_module_28 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_28.parquet")))
metrics = MD.MetricsCallback()

def make_trainer():
  return L.Trainer(
    max_epochs=15,
    max_time={"minutes": 10},
    callbacks=[
      # metrics,
    ],
    deterministic=True,
    default_root_dir=here("cache/lightning"),
    enable_checkpointing=True,
    fast_dev_run=False,
    accelerator="auto", 
  )

trainer = make_trainer()
trainer

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# Distil Bert finetuned (cross entropy loss)

## 3 classes

In [18]:
reload(MD)
distil_3 = MD.DistilBertFinetune(n_emotions=3)
if True:    
  trainer.fit(
    distil_3,
    datamodule=data_module_3,
  )
else:
  distil_3 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_3_14epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_3,
  datamodule=data_module_3,
)
del distil_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss       | BCEWithLogitsLoss                   | 0      | train
3 | f1_standard    | MultilabelF1Score                   | 0      | train
4 | f1_interesting | MultilabelF1Score                   | 0      | train
5 | rmse           | MeanSquaredError                    | 0      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.4887070953845978
   test_expected_ndcg       0.6423622369766235
   test_f1_interesting       0.511014461517334
    test_f1_standard        0.5605985522270203
        test_ndcg            0.923658013343811
        test_rmse           0.2684585750102997
    test_weighted_mse       0.13115161657333374
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.4887070953845978,
  'test_rmse': 0.2684585750102997,
  'test_ndcg': 0.923658013343811,
  'test_expected_ndcg': 0.6423622369766235,
  'test_f1_standard': 0.5605985522270203,
  'test_f1_interesting': 0.511014461517334,
  'test_weighted_mse': 0.13115161657333374}]

## 7 classes

In [19]:
reload(MD)
distil_7 = MD.DistilBertFinetune(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_7,
    datamodule=data_module_7,
  )
else:
  distil_7 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_7_14epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_7,
  datamodule=data_module_7
)
del distil_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.2862784266471863
   test_expected_ndcg       0.4194100499153137
   test_f1_interesting      0.27856186032295227
    test_f1_standard        0.2524093687534332
        test_ndcg           0.8904078006744385
        test_rmse           0.1913556456565857
    test_weighted_mse       0.06533249467611313
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.2862784266471863,
  'test_rmse': 0.1913556456565857,
  'test_ndcg': 0.8904078006744385,
  'test_expected_ndcg': 0.4194100499153137,
  'test_f1_standard': 0.2524093687534332,
  'test_f1_interesting': 0.27856186032295227,
  'test_weighted_mse': 0.06533249467611313}]

## 28 classes

In [20]:
reload(MD)
distil_28 = MD.DistilBertFinetune(n_emotions=28)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_28,
    datamodule=data_module_28,
  )
else:
  distil_28 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_28_11epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.12032134085893631
   test_expected_ndcg       0.27468523383140564
   test_f1_interesting      0.0868084579706192
    test_f1_standard        0.09487314522266388
        test_ndcg           0.8013240694999695
        test_rmse           0.11360271275043488
    test_weighted_mse       0.02303914725780487
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.12032134085893631,
  'test_rmse': 0.11360271275043488,
  'test_ndcg': 0.8013240694999695,
  'test_expected_ndcg': 0.27468523383140564,
  'test_f1_standard': 0.09487314522266388,
  'test_f1_interesting': 0.0868084579706192,
  'test_weighted_mse': 0.02303914725780487}]

In [21]:
# test sample
trainer.predict(
  distil_28,
  ["I Love this"],
)

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[5.0394e-04, 4.9547e-03, 9.6893e-01, 4.3408e-03, 2.3719e-04, 1.1148e-03,
          3.0880e-04, 1.1559e-04, 1.4892e-01, 1.1853e-03, 6.8009e-04, 4.7214e-05,
          1.0245e-03, 7.2380e-04, 1.5844e-02, 1.7071e-03, 4.0053e-02, 5.6471e-05,
          1.3891e-02, 5.1418e-05, 7.4431e-06, 5.4433e-04, 1.0211e-03, 6.8130e-05,
          8.0709e-04, 1.5009e-03, 5.4456e-05, 1.9993e-05]])]

In [22]:
del distil_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')

# Distil Bert finetuned (DCG loss)

## 3 classes

In [23]:
reload(MD) # DistilBertFinetuneOnDCG
distil_dcg_3 = MD.DistilBertFinetuneOnDCG(n_emotions=3)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_3,
    datamodule=data_module_3,
  )
else:
  distil_dcg_3 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_3_10epchs.ckpt"),
    n_emotions=3,
  ) 
out = trainer.test(
  distil_dcg_3,
  datamodule=data_module_3,
)
del distil_dcg_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.6402591466903687
   test_expected_ndcg       0.6344013214111328
   test_f1_interesting      0.3201582729816437
    test_f1_standard        0.22153021395206451
        test_ndcg           0.9250280261039734
        test_rmse           0.35331401228904724
    test_weighted_mse        0.275745689868927
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.6402591466903687,
  'test_rmse': 0.35331401228904724,
  'test_ndcg': 0.9250280261039734,
  'test_expected_ndcg': 0.6344013214111328,
  'test_f1_standard': 0.22153021395206451,
  'test_f1_interesting': 0.3201582729816437,
  'test_weighted_mse': 0.275745689868927}]

## 7 classes

In [24]:
reload(MD)
distil_dcg_7 = MD.DistilBertFinetuneOnDCG(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_7,
    datamodule=data_module_7,
  )
else:
  distil_dcg_7 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_7_10epchs.ckpt"),
    n_emotions=7,
  )
  
out = trainer.test(
  distil_dcg_7,
  datamodule=data_module_7,
)
del distil_dcg_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.44672107696533203
   test_expected_ndcg       0.4190252721309662
   test_f1_interesting      0.17660880088806152
    test_f1_standard        0.40341970324516296
        test_ndcg           0.8842915892601013
        test_rmse           0.3051820695400238
    test_weighted_mse       0.11000015586614609
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.44672107696533203,
  'test_rmse': 0.3051820695400238,
  'test_ndcg': 0.8842915892601013,
  'test_expected_ndcg': 0.4190252721309662,
  'test_f1_standard': 0.40341970324516296,
  'test_f1_interesting': 0.17660880088806152,
  'test_weighted_mse': 0.11000015586614609}]

## 28 classes

In [25]:
reload(MD)
distil_dcg_28 = MD.DistilBertFinetuneOnDCG(n_emotions=28)
trainer = make_trainer()
if True:
  trainer.fit(
    distil_dcg_28,
    datamodule=data_module_28,
  )
else:
  distil_dcg_28 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_28_10epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_dcg_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy        0.508709728717804
   test_expected_ndcg       0.27000129222869873
   test_f1_interesting      0.05060017853975296
    test_f1_standard        0.3457867205142975
        test_ndcg           0.7893139719963074
        test_rmse           0.3792361915111542
    test_weighted_mse       0.14942362904548645
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.508709728717804,
  'test_rmse': 0.3792361915111542,
  'test_ndcg': 0.7893139719963074,
  'test_expected_ndcg': 0.27000129222869873,
  'test_f1_standard': 0.3457867205142975,
  'test_f1_interesting': 0.05060017853975296,
  'test_weighted_mse': 0.14942362904548645}]

In [26]:
# test sample
trainer.predict(
  distil_dcg_28,
  ["I Love this"],
)

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[0.1140, 0.5964, 0.9915, 0.5530, 0.0333, 0.3486, 0.1029, 0.0709, 0.9281,
          0.2287, 0.1114, 0.0508, 0.3300, 0.1668, 0.6178, 0.4084, 0.7820, 0.0306,
          0.6846, 0.0318, 0.0248, 0.1106, 0.1216, 0.0526, 0.2581, 0.6175, 0.1240,
          0.0456]])]

In [27]:
del distil_dcg_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')

# Distil Bert finetuned (Weighed MSE loss)

## 3 classes

In [28]:
reload(MD)
distil_mse_3 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=3)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_3,
    datamodule=data_module_3,
  )
else:
  distil_mse_3 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_3_10epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_mse_3,
  datamodule=data_module_3,
)
del distil_mse_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.49903085827827454
   test_expected_ndcg       0.6393864750862122
   test_f1_interesting      0.4787987768650055
    test_f1_standard        0.6881969571113586
        test_ndcg           0.9288671016693115
        test_rmse           0.27469775080680847
    test_weighted_mse       0.11835694313049316
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.49903085827827454,
  'test_rmse': 0.27469775080680847,
  'test_ndcg': 0.9288671016693115,
  'test_expected_ndcg': 0.6393864750862122,
  'test_f1_standard': 0.6881969571113586,
  'test_f1_interesting': 0.4787987768650055,
  'test_weighted_mse': 0.11835694313049316}]

## 7 classes

In [29]:
reload(MD)
distil_mse_7 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_7,
    datamodule=data_module_7,
  )
else:
  distil_mse_7 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_7_10epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_mse_7,
  datamodule=data_module_7,
)
del distil_mse_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.2991929352283478
   test_expected_ndcg       0.42038047313690186
   test_f1_interesting      0.26331737637519836
    test_f1_standard        0.3369566798210144
        test_ndcg           0.8842048645019531
        test_rmse           0.20385445654392242
    test_weighted_mse       0.06488624215126038
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.2991929352283478,
  'test_rmse': 0.20385445654392242,
  'test_ndcg': 0.8842048645019531,
  'test_expected_ndcg': 0.42038047313690186,
  'test_f1_standard': 0.3369566798210144,
  'test_f1_interesting': 0.26331737637519836,
  'test_weighted_mse': 0.06488624215126038}]

## 28 classes

In [30]:
reload(MD)
distil_mse_28 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=28)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_28,
    datamodule=data_module_28,
  )
else:
  distil_mse_28 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_28_10epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_mse_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy        0.123904287815094
   test_expected_ndcg       0.27411314845085144
   test_f1_interesting      0.10319603234529495
    test_f1_standard        0.1515606939792633
        test_ndcg           0.7988713383674622
        test_rmse           0.11677639931440353
    test_weighted_mse      0.021610287949442863
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.123904287815094,
  'test_rmse': 0.11677639931440353,
  'test_ndcg': 0.7988713383674622,
  'test_expected_ndcg': 0.27411314845085144,
  'test_f1_standard': 0.1515606939792633,
  'test_f1_interesting': 0.10319603234529495,
  'test_weighted_mse': 0.021610287949442863}]

In [31]:
# test sample
trainer.predict(
  distil_mse_28,
  ["I Love this"],
)

Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[3.5239e-03, 1.6392e-02, 9.8768e-01, 8.2411e-04, 2.1077e-03, 6.3017e-04,
          6.7506e-04, 1.2191e-03, 1.3350e-01, 2.5191e-03, 9.6264e-04, 5.4991e-04,
          7.6713e-03, 6.0305e-04, 4.3823e-03, 3.4822e-03, 3.5347e-02, 4.5326e-04,
          2.4907e-02, 1.4430e-04, 2.3408e-05, 2.2928e-03, 1.0280e-03, 9.5675e-05,
          7.3961e-04, 4.4530e-03, 2.7909e-05, 2.0162e-04]])]

In [32]:
del distil_mse_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')